In [1]:
from pathlib import Path
import requests

import numpy as np
import pandas as pd

In [2]:
DATA = Path("data/usgs_earthquakes_2025_m45.csv")
DATA.parent.mkdir(parents=True, exist_ok=True)

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = dict(
    format="csv",
    starttime="2025-01-01",
    endtime="2026-01-01",
    minmagnitude=4.5,
    orderby="time-asc",
)
r = requests.get(url, params=params, timeout=120)
r.raise_for_status()
DATA.write_bytes(r.content)

raw_df = pd.read_csv(DATA, parse_dates=["time"])

raw_df["date"] = raw_df["time"].dt.date
raw_df["month"] = raw_df["time"].dt.month
raw_df["month_name"] = raw_df["time"].dt.strftime("%b")

raw_df["depth_bin"] = pd.cut(
    raw_df["depth"],
    bins=[-1, 10, 30, 70, 300, 700, np.inf],
    labels=["0–10", "10–30", "30–70", "70–300", "300–700", "700+"],
)

raw_df["mag_bin"] = pd.cut(
    raw_df["mag"],
    bins=[-np.inf, 5, 5.5, 6, 6.5, 7, 7.5, 8, np.inf],
    labels=["4.5–5", "5–5.5", "5.5–6", "6–6.5", "6.5–7", "7–7.5", "7.5–8", "8+"],
)

raw_df.to_csv(DATA, index=False)

df = pd.read_csv(DATA, parse_dates=["time"])
print(df.shape)
df.head(2)

(8228, 27)


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,magError,magNst,status,locationSource,magSource,date,month,month_name,depth_bin,mag_bin
0,2025-01-01 01:46:59.538000+00:00,-3.8682,151.6536,10.0,4.7,mb,39.0,99.0,0.557,0.86,...,0.108,26.0,reviewed,us,us,2025-01-01,1,Jan,0–10,4.5–5
1,2025-01-01 02:00:06.918000+00:00,-3.8521,151.6207,10.0,4.5,mb,36.0,98.0,0.591,0.98,...,0.116,22.0,reviewed,us,us,2025-01-01,1,Jan,0–10,4.5–5
